In [0]:
"""
Script escrito em Python com a finalidade de obter os dados do IPCA via da API do Banco Central, armazenamento de DataLake do Databricks por meio da linguagem SQL, permitindo
a conexão com ferramentas de SelfService BI, que neste caso será o Power BI.

O Objetivo é entregar uma ferramenta de análise da inflação cujo processo esteja todo automatizado, permitindo que o demandante 
consiga tomar decisões de forma tempestiva.

O código foi escrito diretamente no notebook da ferramenta Databricks Community Edition, cujos pacotes utilizados já são pré-instalados.

Dependências Python
-------------------
- pandas
- requests
- datetime
- pyspark

Ordem de Execução
-----------------
- Importa os pacotes necessários.
- Define a data inicial e final para a consulta de dados na API.
- Armazena a URL da API na variável url.
- Faz a requisição HTTP para a API e processa a resposta.
- Converte os dados da resposta em um DataFrame do pandas.
- Trata possíveis exceções durante a requisição.
- Cria uma sessão Spark.
- Ajusta os tipos de dados.
- Converte o DataFrame do Pandas para DaraFrame Spark
- Salva os dados internamente via Spark e define o caminho/path.
- Verifica se existe a tabela ipca_delta e caso não ela é criadano caminho/path anterior.
- Realiza consulta simples ao Data Lake para confirmar se a tabela foi criada com sucesso.

"""
# Importação dos pacotes
import pandas as pd
import requests
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format

# Define a data inicial e final para a consulta de dados na API.
hoje = datetime.today()
data_inicial = (hoje - timedelta(days=30*365)).strftime('%d/%m/%Y')
data_final = hoje.strftime('%d/%m/%Y')

# Armazena a URL da API em variável Python
url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.10844/dados?formato=json&dataInicial={data_inicial}&dataFinal={data_final}"

try:
    # Envia a requsição para a API
    resp = requests.get(url)
    resp.raise_for_status()  # Levanta um erro para códigos de status de resposta HTTP diferentes de 200
    dados = resp.json()
    df_ipca = pd.DataFrame(dados)    
    print(df_ipca)
except requests.exceptions.RequestException as e:
    # Informa exceções específicas de requisição HTTP
    print(f"Erro na solicitação HTTP: {e}")
except Exception as e:
    # Informa outras exceções
    print(f"Outro erro ocorreu: {e}")




           data valor
0    01/02/1995  3.09
1    01/03/1995  3.83
2    01/04/1995  4.29
3    01/05/1995  6.91
4    01/06/1995  4.38
..          ...   ...
355  01/09/2024  0.15
356  01/10/2024  0.35
357  01/11/2024  0.83
358  01/12/2024  0.66
359  01/01/2025  0.78

[360 rows x 2 columns]


In [0]:
# Verifica os tipos de dados do DataFrame
df_ipca.info()

In [0]:
# Cria a sessão Spark
spark = SparkSession.builder.appName("IPCA_DeltaLake").getOrCreate()

# Ajusta os tipos de dados no DataFrame
df_ipca["data"] = pd.to_datetime(df_ipca["data"], format= "%d/%m/%Y")
df_ipca["valor"] = df_ipca["valor"].astype(float)

# Converse o DataFrame do Pandas para Spark
df_spark = spark.createDataFrame(df_ipca)

# Define o caminho do Data Lake
delta_path = 'dbfs:/mnt/delta/ipca'

# Salva os arquivos no Data Lake
df_spark.write.format("delta").mode("overwrite").save(delta_path)
print("Dados do IPCA salvos com sucesso no Data Lake")


Dados do IPCA salvos com sucesso no Data Lake
+----------+-----+
|      data|valor|
+----------+-----+
|01/02/1995| 3.09|
|01/03/1995| 3.83|
|01/04/1995| 4.29|
|01/05/1995| 6.91|
|01/06/1995| 4.38|
|01/07/1995|  3.9|
|01/08/1995| 3.31|
|01/09/1995| 2.85|
|01/10/1995| 2.47|
|01/11/1995| 2.71|
|01/12/1995| 2.28|
|01/01/1996|  2.2|
|01/02/1996| 2.32|
|01/03/1996| 1.63|
|01/04/1996|  1.4|
|01/05/1996| 2.21|
|01/06/1996| 1.54|
|01/07/1996| 1.41|
|01/08/1996| 1.43|
|01/09/1996| 1.02|
+----------+-----+
only showing top 20 rows



In [0]:
%sql

-- Verifica se existe a tabela ipca_delta e caso não ela é criada
CREATE TABLE IF NOT EXISTS ipca_delta
USING DELTA
LOCATION 'dbfs:/mnt/delta/ipca'

In [0]:
%sql
-- Consulta para chegagem dos dados armazenados na tabela
SELECT * FROM ipca_delta 
ORDER BY data DESC
LIMIT 10

data,valor
2025-01-01T00:00:00.000+0000,0.78
2024-12-01T00:00:00.000+0000,0.66
2024-11-01T00:00:00.000+0000,0.83
2024-10-01T00:00:00.000+0000,0.35
2024-09-01T00:00:00.000+0000,0.15
2024-08-01T00:00:00.000+0000,0.24
2024-07-01T00:00:00.000+0000,0.75
2024-06-01T00:00:00.000+0000,0.04
2024-05-01T00:00:00.000+0000,0.4
2024-04-01T00:00:00.000+0000,0.05
